In [2]:
pip install optax dm-haiku

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 8.1 MB/s 
     |████████████████████████████████| 350 kB 42.4 MB/s 
     |████████████████████████████████| 85 kB 4.0 MB/s 


Due date: October 21 2022 

# Description
  In this problem we will use apply the LSMC method to price American put options. Specifically, we will replicate the result in the first row, 6th column of Table 1 in [Longstaff and Schwartz 2001](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf)

  

*  Read the introduction of the [paper](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf).
*   We will price an american put option as described in page 126 of the aforementioned article. Read paragraphs 1 and 2 of page 126
* As we saw in class, one of the ways we can use linear regression to fit nonlinear functions is to use polynomial features. A common choice in many applications is to use the so called ``Chebyshev polynomials''. Chebyshev polynomials are defined recursively by:

\begin{equation}
T_0(x) = 1\\
T_1(x) = x\\
T_{n + 1}(x)  =  2 x T_n(x) - T_{n - 1}(x)\\
\end{equation}


# Part 1
The code below simulates the evolution of a stock price that follows a geometric brownian motion. Write a JAX version of that code. You are not allowed to use functions from other libraries. For this part, the "simulate"
function does not need to be jit compiled. As we will see, jit compiling a funciton with for loops may introduce some complications
 

In [3]:
import jax
import jax.numpy as jnp
import optax
from jax import random
from jax import device_put

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


def simulate():
  key = random.PRNGKey(0)

  def step(S):
    dz=random.normal(key,(S.size,)) * jnp.sqrt(dt)
    dz=device_put(dz)
    dS = r * S  * dt + σ  * S  * dz
    S = S + dS
    return S

  S0 = jnp.ones(20000)
  S = S0
  S_list = []

  for t in range(m):
    S = step(S)
    S_list.append(S)

  S_array = jax.numpy.stack(S_list)
  return S_array




In [ ]:

simulate()

DeviceArray([[1.0047877 , 1.0041184 , 1.0012033 , ..., 1.0009823 ,
              0.9972837 , 1.002711  ],
             [1.0095983 , 1.0082538 , 1.0024081 , ..., 1.0019655 ,
              0.9945748 , 1.0054295 ],
             [1.0144318 , 1.0124062 , 1.0036144 , ..., 1.0029497 ,
              0.9918732 , 1.0081553 ],
             ...,
             [1.5969114 , 1.4959735 , 1.1250849 , ..., 1.1009947 ,
              0.7660111 , 1.3038602 ],
             [1.6045569 , 1.5021346 , 1.1264387 , ..., 1.1020762 ,
              0.76393044, 1.3073951 ],
             [1.612239  , 1.508321  , 1.1277943 , ..., 1.1031587 ,
              0.76185536, 1.3109396 ]], dtype=float32)

# Part 2
Write a jit compiled version of the simulate function. You may want to check out the function jax.lax.scan.


In [7]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100

def simulate(x,S):
  x=0
  return  x,S+ r * S  * dt + σ  * S  * random.normal(key,(S.size,)) * jnp.sqrt(dt)
key = random.PRNGKey(0)
S0 = jnp.ones((20000,20000)) 
S = S0
fast_s = jax.jit(simulate)
jax.lax.scan(simulate,0,S)[1]


DeviceArray([[1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ],
             [1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ],
             [1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ],
             ...,
             [1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ],
             [1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ],
             [1.0047877 , 1.0041184 , 1.0012034 , ..., 1.0009823 ,
              0.99728376, 1.0027112 ]], dtype=float32)

# Part 3
The code below computes the price of an American Put option using Least Squares Monte Carlo (LSMC). Write a JAX version of that code. You are not allowed to use functions from other libraries. Your "compute_price" function must be jit compiled.

In [5]:
from jax._src import array
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
from jax import device_put

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates


# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x):

  def chebyshev_step(B1,x):
    Bn = 2 * x * B1[1] - B1[0]
    Bn1 = jnp.vstack((B1[1],Bn))
    return Bn1, Bn
 
  B0 = jnp.array([jnp.ones(len(x)), x])
  x0 = jnp.zeros((order-2,len(x)))
  x1 = x0.at[:].add(x)
  B = jnp.transpose(jnp.vstack((B0,jax.lax.scan(chebyshev_step, B0, x1)[1])))

  return B

# scales x to be in the interval(-1, 1)
# @jax.jit
def scale(x):
  xmin = x.min()
  xmax = x.max()
  a = 2 / (xmax - xmin)
  b = 1 - a * xmax
  return a * x + b

# simulates one step of the stock price evolution
# @jax.jit
def step(S,dz):
  dB = jnp.sqrt(Δt) * dz
  S_tp1 = S + r * S * Δt + σ * S * dB
  return S_tp1,S_tp1

# @jax.jit
def payoff_put(S):
  return jnp.maximum(K - S, 0.)

# LSMC algorithm
# @jax.jit
def compute_price():
  key=random.PRNGKey(2)
  S0 = Spot * jnp.ones(n)
  dz = random.normal(key,shape=(m,n))
  S1 = jax.lax.scan(step,S0,dz)[1]
  S = jnp.vstack((S0,S1))

  # for in range(m):
  # S_tp1 = step(S[t])
  # S.append(S_tp1)
  discount = jnp.exp(-r * Δt)

  # Very last date
  value_if_exercise = payoff_put(S[-1])
  discounted_future_cashflows = value_if_exercise * discount

  # Proceed recursively
  def simu_dfc(dfc,s):
    X = chebyshev_basis(scale(s))
    Y = dfc

    Θ = jnp.linalg.solve(X.T @ X, X.T @ Y)
    value_if_wait = X @ Θ
    value_if_exercise = payoff_put(s)
    exercise = value_if_exercise >= value_if_wait
    dfc = discount * jnp.where(
        exercise,
        value_if_exercise,
        dfc)
    return dfc, dfc
  
  price=jax.lax.scan(simu_dfc,discounted_future_cashflows,S[-2:-m-1：-1])[0]
  return price.mean()

print(compute_price())



4.0618916


DeviceArray([[1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633],
             [1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633],
             [1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633],
             ...,
             [1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633],
             [1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633],
             [1.0343468, 1.0296146, 1.0090019, ..., 1.0074385, 0.9812858,
              1.0196633]], dtype=float32)